In [1]:
import flask

import numpy as np
import pandas as pd
import pickle
from collections import defaultdict

from sklearn.neighbors import NearestNeighbors



from fuzzywuzzy import process
from fuzzywuzzy import fuzz



In [2]:
with open('./data/df_merged_cleaned.pkl', 'rb') as picklefile:
    df_merged = pickle.load(picklefile)

with open('./data/topic_names.pkl', 'rb') as picklefile:
    topic_names = pickle.load(picklefile)

with open('./data/lda_model.pkl', 'rb') as picklefile:
    lda_mod = pickle.load(picklefile)

with open('./data/vectorizer.pkl', 'rb') as picklefile:
    vect_mod = pickle.load(picklefile)

with open('./data/lda_model_data.pkl', 'rb') as picklefile:
    lda_data = pickle.load(picklefile)

cleaned_talks = df_merged['text']

titles = df_merged['title']

In [3]:
app = flask.Flask(__name__)

In [ ]:
def get_recommendations(first_article, model, vectorizer, training_vectors,title, ind):

    new_vec = model.transform(
        vectorizer.transform([first_article]))

    nn = NearestNeighbors(n_neighbors=10, metric='cosine', algorithm='brute')
    nn.fit(training_vectors)
    results = nn.kneighbors(new_vec)
    recommend_list = results[1][0]
    scores = results[0]

    rec_dict = defaultdict(list)
    urls = []
    ss = np.array(scores).flat
    
    print(recommend_list)
#     for i, resp in enumerate(recommend_list):
        
#         rec_dict["0"].append(df_merged.iloc[resp,1])
        
# #         rec_dict['0'].append(df_merged.iloc[i]['title'])
# #         rec_dict['0'].append(df_merged.iloc[i]['url'])
# #         rec_dict['0'].append(df_merged.iloc[i]['title'])
        
# #         rec_dict["desc"].append(df_merged.iloc[resp,1])
# #         rec_dict["url"].append(df_merged.iloc[i]['url'])
# #         rec_dict["title"].append(df_merged.iloc[i]['title'])
# #         urls.append(df_merged.iloc[i]['url'])
# #         rec_dict['title'] = df_merged[title]
    
#     for i, resp in enumerate(recommend_list):
#         rec_dict['0'].append(df_merged.iloc[i]['url'])

    for i in recommend_list:
        rec_dict['0'].append(df_merged.iloc[i]['title'])
        
    for i in recommend_list:
        rec_dict['0'].append(df_merged.iloc[i]['url'])
        
        
    return rec_dict

In [ ]:
get_recommendations(cleaned_talks[7], lda_mod, vect_mod,
                                       lda_data, 'Do schools kill creativity?', 1)

[   7 2037  359 1529 1748  358 2385 1636 1851 1522]


defaultdict(list,
            {'0': ["Behind the design of Seattle's library",
              'Why great architecture should tell a story',
              'Ways of seeing',
              'Architecture at home in its community',
              'How to reinvent the apartment building',
              'Organic algorithms in architecture',
              'Why glass towers are bad for city life -- and what we need instead',
              'How public spaces make cities work',
              'Why the buildings of the future will be shaped by ... you',
              'Ingenious homes in unexpected places',
              'https://www.ted.com/talks/joshua_prince_ramus_on_seattle_s_library\n',
              'https://www.ted.com/talks/ole_scheeren_why_great_architecture_should_tell_a_story\n',
              'https://www.ted.com/talks/rob_forbes_on_ways_of_seeing\n',
              'https://www.ted.com/talks/xavier_vilalta_architecture_at_home_in_its_community\n',
              'https://www.ted.com/talks/m

In [ ]:
@app.route("/")
def visualize_page():

    
    with open("ted_html.html", 'r') as viz_file:
        return viz_file.read()

In [ ]:
@app.route("/ted", methods=["POST"])
def ted():
    """
    When A POST request with json data is made to this uri,
    Read the example from the json, predict probability and
    send it with a response
    """
    # Get decision score for our example that came with the request
    data = flask.request.json
    print(data)
    X = data#["example"])
    #X = str(X[0])
    title, score, talk_ind = process.extractOne(X, titles, scorer=fuzz.token_set_ratio)
    recs = get_recommendations(cleaned_talks[talk_ind],lda_mod, vect_mod, lda_data, title,talk_ind)
#     recs, urls = get_recommendations(cleaned_talks[talk_ind],lda_mod, vect_mod, lda_data, title,talk_ind)
    print(recs.values())
    # Put the result in a nice dict so we can send it as json
    #results = {"recommends": recs[1]}
    #print(results)
    
    return flask.jsonify(recs['0'])

    

#     return flask.jsonify(recs['0']), flask.jsonify(urls)
#     return flask.jsonify(recs['0']), flask.jsonify(recs['1'])

#     print(flask.jsonify(descriptions = recs['desc'], titles = recs['title'], urls = recs['urls']))
    
#     return flask.jsonify(descriptions = recs['desc'], titles = recs['title'], urls = recs['urls'])

#     d = {'descriptions' : recs['desc'], 'titles' : recs['title'], 'urls' : recs['urls']}
#     message = {
#         'status': 200,
#         'message': 'OK',
#         'scores': d
#     }
#     resp = flask.jsonify(message)
#     resp.status_code = 200
#     return resp
app.run(host='0.0.0.0')
app.run(debug=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [26/May/2019 17:59:49] "GET / HTTP/1.1" 200 -
/anaconda3/lib/python3.7/site-packages/sklearn/decomposition/online_lda.py:395: UserWarning: Loky-backed parallel loops cannot be nested below threads, setting n_jobs=1
  n_jobs = effective_n_jobs(self.n_jobs)
127.0.0.1 - - [26/May/2019 17:59:52] "POST /ted HTTP/1.1" 200 -


schools
[   0  406 1342 1354 1120 1372 2416 1723 2247 1968]
dict_values([['Do schools kill creativity?', 'The difference between winning and succeeding', 'Kids need structure', 'Dare to educate Afghan girls', 'Unlock the intelligence, passion, greatness of girls', 'A girl who demanded school', 'Can art amend history?', 'Why lunch ladies are heroes', 'Help for kids the education system ignores', 'How I stopped the Taliban from shutting down my school', 'https://www.ted.com/talks/ken_robinson_says_schools_kill_creativity\n', 'https://www.ted.com/talks/john_wooden_on_the_difference_between_winning_and_success\n', 'https://www.ted.com/talks/colin_powell_kids_need_structure\n', 'https://www.ted.com/talks/shabana_basij_rasikh_dare_to_educate_afghan_girls\n', 'https://www.ted.com/talks/leymah_gbowee_unlock_the_intelligence_passion_greatness_of_girls\n', 'https://www.ted.com/talks/kakenya_ntaiya_a_girl_who_demanded_school\n', 'https://www.ted.com/talks/titus_kaphar_can_art_amend_history\n', 'h

/anaconda3/lib/python3.7/site-packages/sklearn/decomposition/online_lda.py:395: UserWarning: Loky-backed parallel loops cannot be nested below threads, setting n_jobs=1
  n_jobs = effective_n_jobs(self.n_jobs)
127.0.0.1 - - [26/May/2019 18:00:04] "POST /ted HTTP/1.1" 200 -


goal
[ 762 1456 1942 2160 2043 1051 1540  772 1390 1443]
dict_values([['Global power shifts', 'A tale of two political systems', 'The surprising way groups like ISIS stay in power', 'Why Brexit happened -- and what to do next', 'Capitalism will eat democracy -- unless we speak up', 'The global power shift', 'Is China the new idol for emerging economies?', 'The path to ending ethnic conflicts', 'We the People, and the Republic we must reclaim', 'Imagine a European democracy without borders', 'https://www.ted.com/talks/joseph_nye_on_global_power_shifts\n', 'https://www.ted.com/talks/eric_x_li_a_tale_of_two_political_systems\n', 'https://www.ted.com/talks/benedetta_berti_the_surprising_way_groups_like_isis_stay_in_power\n', 'https://www.ted.com/talks/alexander_betts_why_brexit_happened_and_what_to_do_next\n', 'https://www.ted.com/talks/yanis_varoufakis_capitalism_will_eat_democracy_unless_we_speak_up\n', 'https://www.ted.com/talks/paddy_ashdown_the_global_power_shift\n', 'https://www.ted.

...

religion
[   8 1403 1686 1178  931 1687 1059 2325 2313 1807]
dict_values([["Let's teach religion -- all religion -- in schools", 'Txtng is killing language. JK!!!', 'Is religion good or bad? (This is a trick question)', "Why is 'x' the unknown?", 'A Rosetta Stone for a lost language', 'What makes a word "real"?', 'Atheism 2.0', 'Should we simplify spelling?', 'What I learned from 2,000 obituaries', 'Go ahead, make up new words!', 'https://www.ted.com/talks/dan_dennett_s_response_to_rick_warren\n', 'https://www.ted.com/talks/john_mcwhorter_txtng_is_killing_language_jk\n', 'https://www.ted.com/talks/kwame_anthony_appiah_is_religion_good_or_bad_this_is_a_trick_question\n', 'https://www.ted.com/talks/terry_moore_why_is_x_the_unknown\n', 'https://www.ted.com/talks/rajesh_rao_computing_a_rosetta_stone_for_the_indus_script\n', 'https://www.ted.com/talks/anne_curzan_what_makes_a_word_real\n', 'https://www.ted.com/talks/alain_de_botton_atheism_2_0\n', 'https://www.ted.com/talks/karina_galperin_

/anaconda3/lib/python3.7/site-packages/sklearn/decomposition/online_lda.py:395: UserWarning: Loky-backed parallel loops cannot be nested below threads, setting n_jobs=1
  n_jobs = effective_n_jobs(self.n_jobs)
127.0.0.1 - - [26/May/2019 18:03:54] "POST /ted HTTP/1.1" 200 -


leadership
[ 519 2158  176  627  714  639  831  991 1144  241]
dict_values([['Tribal leadership', '"Redemption Song"', '"Tembererana"', 'How to start a movement', 'The Axis of Evil Middle East Comedy Tour', 'Homeopathy, quackery and fraud', "A whistleblower you haven't heard", 'Trusting the ensemble', 'Gaming for understanding', 'The transformative power of classical music', 'https://www.ted.com/talks/david_logan_on_tribal_leadership\n', 'https://www.ted.com/talks/john_legend_redemption_song\n', 'https://www.ted.com/talks/raul_midon_plays_all_the_answers_and_tembererana\n', 'https://www.ted.com/talks/derek_sivers_how_to_start_a_movement\n', 'https://www.ted.com/talks/jamil_abu_wardeh_bringing_comedy_to_the_axis_of_evil\n', 'https://www.ted.com/talks/james_randi\n', 'https://www.ted.com/talks/a_whistleblower_you_haven_t_heard\n', 'https://www.ted.com/talks/charles_hazlewood\n', 'https://www.ted.com/talks/brenda_brathwaite_gaming_for_understanding\n', 'https://www.ted.com/talks/benjamin_

/anaconda3/lib/python3.7/site-packages/sklearn/decomposition/online_lda.py:395: UserWarning: Loky-backed parallel loops cannot be nested below threads, setting n_jobs=1
  n_jobs = effective_n_jobs(self.n_jobs)
127.0.0.1 - - [26/May/2019 18:04:48] "POST /ted HTTP/1.1" 200 -


juggling
[ 222  187  304 1037  270 1479  841   88 1230  256]
dict_values([['Juggling as art ... and science', 'The untouchable music of the theremin', 'Playing invisible turntables', 'There are no mistakes on the bandstand', 'Close-up card magic with a twist', 'The polyphonic me', 'A modern take on piano, violin, cello', 'How to truly listen', 'The mad scientist of music', 'Brain magic', 'https://www.ted.com/talks/michael_moschen_juggles_rhythm_and_motion\n', 'https://www.ted.com/talks/pamelia_kurstin_plays_the_theremin\n', 'https://www.ted.com/talks/james_burchfield_plays_invisible_turntables\n', 'https://www.ted.com/talks/stefon_harris_there_are_no_mistakes_on_the_bandstand\n', 'https://www.ted.com/talks/lennart_green_does_close_up_card_magic\n', 'https://www.ted.com/talks/beardyman_the_polyphonic_me\n', 'https://www.ted.com/talks/ahn_trio_a_modern_take_on_piano_violin_cello\n', 'https://www.ted.com/talks/evelyn_glennie_shows_how_to_listen\n', 'https://www.ted.com/talks/mark_applebau